In [1]:
%cd ..

/home/dani2442/Projects/bifurcationjax


/home/dani2442/Projects/bifurcationjax/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import jax.numpy as jnp
import jax

import matplotlib.pyplot as plt

from bifurcationjax.continuation.Corrector import CorrectorParams
from bifurcationjax.continuation.Predictor import PredictorParams
from bifurcationjax.continuation.Continuation import continuation
from bifurcationjax.BifurcationProblem import BifurcationProblem
from bifurcationjax.utils.Branch import ContinuationPar
from bifurcationjax.utils.plot import plot_bifurcation_diagram
from bifurcationjax.utils.bifurcation import get_bifurcation_type, is_stable
from bifurcationjax.continuation.Corrector import CorrectorParams, NaturalCorrector
from bifurcationjax.utils.Branch import Point


In [3]:
@jax.jit
def F(x, mu):
    return x*(mu - x)

def plot_fn(p):
    return p.z[0]

p0 = -0.2
x0 = jnp.array([0.])

prob = BifurcationProblem(F, x0, p0,)
dz0 = jnp.array([0.,1.])

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [4]:
prob = BifurcationProblem(F, x0, p0,)
par = ContinuationPar(p_min=-1., p_max=1., dsmax=0.0001, max_steps=500)
prediction_params = PredictorParams(method='tangent', k=1, dz0=dz0)
correction_params = CorrectorParams(method='PALC', epsilon=1e-3)

In [5]:
p0 = Point(step=0)
J = jax.jit(jax.jacobian(prob.f))
z = jnp.append(prob.x0, prob.p0)

p0.z, success = NaturalCorrector(delta=0.2, k=0)(z, None, prob.f, None, par.dsmax)

p0.evals, p0.evecs = jnp.linalg.eig(J(p0.z[:-1], p0.z[-1])) 
p0.n_unstable, p0.n_imag, p0.stable = is_stable(p0.evals)

predictor = prediction_params.init()
corrector = correction_params.init()

In [6]:
zpred, v = predictor(p0.z, par.dsmax, prob.f)

In [9]:
%%timeit
zpred, v = predictor(p0.z, par.dsmax, prob.f)

128 µs ± 1.24 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
%%timeit
z, success = corrector(zpred, p0.z, prob.f, v, par.dsmax)

268 µs ± 6.65 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
